# Kirbo Bot
---
Written, and developed by cataclysm-interactive.
Kirbo uses the following libraries.

In [8]:
from ctypes import util
import json
import os
import ast
import re
from warnings import resetwarnings
from discord import colour, embeds, member, user
from discord.ext.commands.converter import MemberConverter
import requests
import discord
import random
import asyncio
from dotenv import load_dotenv
from discord.ext import commands
from discord import Color
from datetime import datetime, timedelta
from datetime import date
from discord.utils import get
from requests import api
from requests.models import Response

VERSION = 'V0.1.7 Factions'

## What is this?
---
This is a Jupyter notebook that shows how Kirbo was developed, and serves as a sort of documentation for the bot.

!!PLEASE NOTE!!

Running the code in this notebook WILL NOT work! Since it's a discord bot, that tries to run forever, and the notebook cuts it off after the last line is executed. aka: no, this notebook is not a bot in a book.

### What does Kirbo do?
---
Kirbo does a few different things. 
1. Kirbo has a few moderation commands, most notably mute.
2. Kirbo has an income system that allows people to collect and spend a virtual currency

## Enough blabbing, Let's see your code!
---
Ok fine.
First, lets initalize the variables that are used in Kirbo's main operations.
Kirbo starts by getting the tokens and ID's that are used to connect to Discord, and the CataclysmAPI.
It also sets the bot's status to "Listening to Army Gang"

In [9]:
load_dotenv()
TOKEN = os.getenv('DISCORD_TOKEN')
GUILD = os.getenv('DISCORD_GUILD')
PINK = Color.from_rgb(255,185,209)


USER_URL="https://cataclysmapi20211218110154.azurewebsites.net/api/users"

FACTION_URL="https://cataclysmapi20211218110154.azurewebsites.net/api/factions"

intents = discord.Intents.default()
intents.members = True
bot = commands.Bot(command_prefix=',',intents=intents,activity=discord.Activity(type=discord.ActivityType.listening, name='Army Gang!'))
bot.titan=None

## Helper Functions
---
Next, we have a few helper functions I made to help with a few things. (duh)

on_ready() is a the function that is called once the bot has actually connected to discord. It also sets two bot variables, one is a user ID so the bot can DM me. The other is the admin chat in Army Gang so that the bot can send messages there.

getAllUsers() Does exactly what it sounds like it does. It gets all the registered users from the CataclysmAPI.

getUserData(id) gets a certain user from the CataclysmAPI using the user's discord ID.

getAllFactions() gets all the factions from the CataclysmAPI

getUserFaction(id) gets a certain user's faction using the user's ID.

updateFaction(faction) sends an updated faction to the CataclysmAPI.

These functions are VERY helpful when it comes to managing users in the API.

In [10]:
@bot.event
async def on_ready():
    bot.titan = bot.get_user(847989667088564244)
    bot.adminChat = bot.get_channel(764867760957947934)
    print(f'{bot.user.name} has connected to Discord!')

# Helper Functions -----------------------------------------------------------------
def getAllUsers():
    response = requests.get(USER_URL)
    if response.status_code != 200:
        return None
    else:
        jsonResponse = ast.literal_eval(str(response.json()))
        return jsonResponse

def getUserData(id):
    url = USER_URL + "/" + str(id)
    response = requests.get(url)
    if response.status_code != 200:
        return None
    else:
        jsonResponse = ast.literal_eval(str(response.json()))
        return jsonResponse

def getAllFactions():
    response = requests.get(FACTION_URL)
    if response.status_code != 200:
        return None
    else:
        jsonResponse = ast.literal_eval(str(response.json()))
        return jsonResponse

def getUserFaction(id):
    response = getAllFactions()
    for x in response:
        factionData = ast.literal_eval(str(x))
        if ',' in factionData["factionMembers"]:
            members = str(factionData["factionMembers"]).split(",")
            for y in members:
                if y == str(id):
                    return factionData
        else:
            if factionData["factionMembers"] == str(id):
                return factionData

def updateFaction(faction):
    json = {
        "id":faction["id"],
        "factionName":faction["factionName"],
        "factionIncome":faction["factionIncome"],
        "factionMembers":faction["factionMembers"],
        "factionLandClaim":faction["factionLandClaim"],
        "factionLogo":faction["factionLogo"],
        "attack":faction["attack"],
        "defense":faction["defense"],
        "utility":faction["utility"],
        "balance":faction["balance"]
        }
    print(json)
    response = requests.put(FACTION_URL+"/"+str(faction["id"]),json=json)
    return response

## User Income
---
This function was a pain in the butt to get working properly.

Overall, its pretty simple, its a function that is scheduled to run every hour, and gets all the users that are in a faction, and adds the faction income to their token balance.

In [11]:
# User Income Update Function ----------------------------------------------
from apscheduler.schedulers.background import BackgroundScheduler

# Start the scheduler
sched = BackgroundScheduler()

def job_function():
    users = getAllUsers()
    for user in users:
        faction = getUserFaction(user["id"])
        if faction != None:
            amount = int(faction["factionIncome"])
            tokens = int(user["token"]) + amount
            jsonData = {"id": user["id"], "token": tokens, "date": f"{user['date']}"}
            print(jsonData)
            response = requests.put(USER_URL + "/" + str(user["id"]),json=jsonData)
            if(response.status_code != 204):
                print(f"Something went wrong with adding Tokens to user's balance. Heres the error code: {response.status_code}")
         

# Schedules job_function to be run once each hour
minutes = 1
hours = 1
sched.add_job(job_function, 'interval', hours=hours)
sched.start()

## The Help Command
---
Ah yes, the humble help command. The first command that people use when looking to use a discord bot.

Kirbo's is no different, and it lists (almost) all the commands that Kirbo has.
Its also the first command that my friends helped me test.

Pretty simple, makes a discord embed object, and adds all the commands to it, and prints it to the chat.

In [12]:
#Help Command --------------------------------------------------------------
bot.remove_command('help')
@bot.command("help")
async def help(ctx, comnd: str=None):
    embed=discord.Embed(title="Kirbo Help",description="This bot's prefix is ','\nSome of the commands have aliases.",color=PINK)
    embed.add_field(name="Fun Commands", value="about, poyo, roll, slap, shoot",inline=False)
    embed.add_field(name="Economy Commands",value="balance, daily, store, buy, give",inline=False)
    embed.add_field(name="Faction Commands",value="faction, createfaction, leavefaction, invite, deposit, factionstore",inline=False)
    embed.add_field(name="Debugging Commands",value="testapi, shutdown",inline=False)

    await ctx.send(embed=embed)

## Connecting to Discord
---
Yep. Thats all this singular line does.

In [ ]:
bot.run(TOKEN)

# Full, Uninterupted Kirbo Code is in bot.py.